In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import sys
sys.path.append("/global/homes/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/LightningModules/TrackML_ACAT")
import cuml
import torch
event = torch.load("/global/cfs/cdirs/m3443/data/trackml-codalab/gnn_processed/1GeV_pt_cut_quickstart_example/test/1011")
from tracking_utils import eval_metrics
from torch_geometric.data import Data
import yaml
sys.path.append('../..')
from LightningModules.TrackML_ACAT.GNNEmbedding.Models.models import HierarchicalGNN, InteractionGNN

event = Data.from_dict(event.__dict__)
event.scores = (event.scores[:len(event.scores)//2] + event.scores[len(event.scores)//2:])/2
bipartite_graph = torch.stack([torch.arange(len(event.pid), device = "cuda"), event.pid], dim = 0).long()

In [11]:
eval_metrics(bipartite_graph, event, pt_cut =1., primary = False)

{'track_eff': 1.0, 'track_pur': 1.0, 'fake_rate': 0.0, 'particle_eff': 1.0}

In [13]:
import cudf
import cugraph
import cupy as cp

In [42]:
def track_candidate_generation(event, score_cut, minimum_spanning_tree, truth = False):
    G = cugraph.Graph()
    if truth:
        df = cudf.DataFrame({"src": cp.asarray(event.edge_index[0][event.pid[event.edge_index[0]]==event.pid[event.edge_index[1]]]),
                             "dst": cp.asarray(event.edge_index[1][event.pid[event.edge_index[0]]==event.pid[event.edge_index[1]]]),
                             "weights": 1 - cp.asarray(event.scores[event.pid[event.edge_index[0]]==event.pid[event.edge_index[1]]])})
    else:
        df = cudf.DataFrame({"src": cp.asarray(event.edge_index[0]),
                             "dst": cp.asarray(event.edge_index[1]),
                             "weights": 1 - cp.asarray(event.scores)})
    G.from_cudf_edgelist(
        df,
        source = "src",
        destination = "dst",
        edge_attr = "weights"
    )
    
    if minimum_spanning_tree: 
        df = cugraph.tree.minimum_spanning_tree(G, weight="weights").view_edge_list()
    G = cugraph.Graph()
    if truth:
        pass
    else:
        df = df[df["weights"] < 1 - score_cut]
    G.from_cudf_edgelist(df,source = "src", destination = "dst", edge_attr = "weights")
    connected_components = cugraph.components.connected_components(G)
    bipartite_graph = torch.stack([torch.as_tensor(connected_components["vertex"]), torch.as_tensor(connected_components["labels"])], dim = 0)
    return eval_metrics(bipartite_graph, event, pt_cut =1., primary = False, nhits_cut = 5)

In [16]:
track_candidate_generation(event, 0.8, False)

{'track_eff': 0.8806682577565632,
 'track_pur': 0.9598671723267292,
 'fake_rate': 0.23018080667593876,
 'particle_eff': 0.7897668661027439}

In [43]:
def get_statistics(truth = False):
    with open("GNN_embedding_default.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)
        
    if hparams["use_toy"]:
        hparams["regime"] = []
        hparams["spatial_channels"] = 2

    if hparams["model"] == "HGNN":
        model = HierarchicalGNN(hparams)
    if hparams["model"] == "IGNN":
        model = InteractionGNN(hparams)

    model.setup("test")
    results = None
    for event in model.test_dataloader():
        event.scores = (event.scores[:len(event.scores)//2] + event.scores[len(event.scores)//2:])/2
        if results is None:
            results = track_candidate_generation(event, 0.8, False, truth)
        else:
            new_results = track_candidate_generation(event, 0.8, False, truth)
            for name in results.keys():
                results[name] += new_results[name]
    for name in results.keys():
        results[name] = results[name]/len(model.test_dataloader())
    return results

In [44]:
results = get_statistics(truth = True)

In [45]:
results

{'track_eff': 0.999080137597702,
 'track_pur': 1.0,
 'fake_rate': 0.04720080204789033,
 'particle_eff': 0.9754255187821436}